# Grade prediction with BERT embedding

In this notebook, we will use pre-trained deep learning model to process some text. We will then use the output of that model to classify the reviews we scrapped in previous courses. We will try to predict wether it was positive or negative. 

Under the hood, the model is actually made up of two model.


1.   DistilBERT processes the sentence and passes along some information it extracted from it on to the next model. DistilBERT is a smaller version of BERT developed and open sourced by the team at HuggingFace. It’s a lighter and faster version of BERT that roughly matches its performance.
2.   The next model, a basic Logistic Regression model from scikit learn will identify if the review was positive or negative



## Installing the transformers library

Let's start by installing the huggingface transformers library so we can load our deep learning NLP model.

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 41.3 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

## Importing the dataset

We'll import the dataset as we did in previous notebooks:

In [3]:
from google.colab import drive
import os
drive.mount('/content/drive')
dirpath = "drive" 
os.listdir(dirpath)

Mounted at /content/drive


['MyDrive', '.shortcut-targets-by-id', '.file-revisions-by-id', '.Trash-0']

In [4]:
import os
reviews = pd.read_csv('/content/drive/MyDrive/reviews.csv')

# IMPORT YOUR REVIEWS FILE





In [5]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37294 entries, 0 to 37293
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       37294 non-null  int64 
 1   page             37294 non-null  int64 
 2   titre            37294 non-null  object
 3   verbatim         36314 non-null  object
 4   date             37294 non-null  object
 5   note             37294 non-null  int64 
 6   reponse          7972 non-null   object
 7   date_experience  37294 non-null  object
 8   fournisseur      37294 non-null  object
 9   source           37294 non-null  object
dtypes: int64(3), object(7)
memory usage: 2.8+ MB


In the first place we should determine what are we considering a positive review. In this case we shall consider as positive every comment that had 4 or 5 as grade and negative otherwise.

In [6]:
# ASSIGN A LABEL TO EACH REVIEW TO DIFFERENCITATE POSITIVE AND NEGATIVE REVIEWS
def get_sentiment(note):
    if note > 3:
        return 'positive'
    else :
        return 'negative'

# apply function to create new column
reviews['label'] = reviews['note'].apply(get_sentiment)

# REMOVE REVIEWS WITH NULL VALUES
reviews.dropna()

,Unnamed: 0,page,titre,verbatim,date,note,reponse,date_experience,fournisseur,source,label
1,1,1,Engie facture a ses clients des sommes…,Engie facture a ses clients des sommes exorbit...,Il y a un jour,1,"Bonjour Julien Blanco,\n\nPour des raisons de ...",Date de l'expérience: 26 novembre 2022,https://fr.trustpilot.com/review/engie.fr,trustpilot,negative
2,2,1,Facturation sur consommation d'un autre logement,Ils me facturent sur le pdl du logement au des...,ll y a 3 jours,1,"Bonjour BlooDz,\n\nPour des raisons de confide...",Date de l'expérience: 29 novembre 2022,https://fr.trustpilot.com/review/engie.fr,trustpilot,negative
3,3,1,un service client ou il est dur de…,un service client ou il est dur de comprendre ...,ll y a 3 jours,1,"Bonjour Ricanto77,\nPour des raisons de confid...",Date de l'expérience: 29 novembre 2022,https://fr.trustpilot.com/review/engie.fr,trustpilot,negative
5,5,1,Service commercial déplorable dont on…,Service commercial déplorable dont on ne compr...,ll y a 4 jours,1,"Bonjour Dubois,\n\nPour des raisons de confide...",Date de l'expérience: 28 novembre 2022,https://fr.trustpilot.com/review/engie.fr,trustpilot,negative
6,6,1,En cours de litige actuellement j…,En cours de litige actuellement j attends des ...,ll y a 5 jours,1,"Bonjour Pierre “juniorB”,\nPour des raisons de...",Date de l'expérience: 17 octobre 2022,https://fr.trustpilot.com/review/engie.fr,trustpilot,negative
...,...,...,...,...,...,...,...,...,...,...,...
37235,37235,503,Avis client,comme toujours l'on ne comprend pas tout du pr...,le 01/03/2022 par WILLIAM D.,2,"Monsieur,",suite à une expérience du 27/01/2022,https://www.avis-verifies.com/avis-clients/edf...,avis_verifies,negative
37236,37236,503,Avis client,on ne sait pas tout avec un seul RDV,le 01/03/2022 par WILLIAM D.,3,"Bonjour Monsieur,",suite à une expérience du 27/01/2022,https://www.avis-verifies.com/avis-clients/edf...,avis_verifies,negative
37237,37237,503,Avis client,TROP CHERS MAIS SYMPAS,le 01/03/2022 par Claire M.,3,"Bonjour Madame,",suite à une expérience du 02/02/2022,https://www.avis-verifies.com/avis-clients/edf...,avis_verifies,negative
37267,37267,506,Avis client,Demarche globale frauduleuse comme beaucoup d'...,le 27/02/2022 par Jean pierre M.,1,"Bonjour Monsieur,",suite à une expérience du 25/01/2022,https://www.avis-verifies.com/avis-clients/edf...,avis_verifies,negative


For computational purposes, we'll only use 1000 sentences which have less than 100 words in it.

In [7]:
# Filter the reviews
# define function to count words in a string
def count_words(value):
    if isinstance(value, str):
        words = value.split()
        return len(words)
    else:
        return 101

# filter reviews to keep only rows with less than 100 words in 'verbatim' column
reviews = reviews[reviews['verbatim'].apply(count_words) < 100]
reviews = reviews.head(1000)

In [8]:
reviews

,Unnamed: 0,page,titre,verbatim,date,note,reponse,date_experience,fournisseur,source,label
0,0,1,Aucun soucis particulier,Je paie ma facture tous les deux mois en fonct...,Il y a 17 heures,4,NaN,Date de l'expérience: 01 décembre 2022,https://fr.trustpilot.com/review/engie.fr,trustpilot,positive
1,1,1,Engie facture a ses clients des sommes…,Engie facture a ses clients des sommes exorbit...,Il y a un jour,1,"Bonjour Julien Blanco,\n\nPour des raisons de ...",Date de l'expérience: 26 novembre 2022,https://fr.trustpilot.com/review/engie.fr,trustpilot,negative
2,2,1,Facturation sur consommation d'un autre logement,Ils me facturent sur le pdl du logement au des...,ll y a 3 jours,1,"Bonjour BlooDz,\n\nPour des raisons de confide...",Date de l'expérience: 29 novembre 2022,https://fr.trustpilot.com/review/engie.fr,trustpilot,negative
3,3,1,un service client ou il est dur de…,un service client ou il est dur de comprendre ...,ll y a 3 jours,1,"Bonjour Ricanto77,\nPour des raisons de confid...",Date de l'expérience: 29 novembre 2022,https://fr.trustpilot.com/review/engie.fr,trustpilot,negative
4,4,1,Client d'ENGIE depuis longtemps toujours satis...,Excellente expérience avec ENGIE et une interl...,Il y a 24 minutes,5,NaN,Date de l'expérience: 01 décembre 2022,https://fr.trustpilot.com/review/engie.fr,trustpilot,positive
...,...,...,...,...,...,...,...,...,...,...,...
1472,1472,16,Rapidité a repondre a nos question et…,Rapidité a repondre a nos question et au télép...,25 juil. 2022,5,"Bonjour,\nMerci pour le détail de votre retour...",Date de l'expérience: 25 juillet 2022,https://fr.trustpilot.com/review/ohm-energie.com,trustpilot,positive
1473,1473,16,Service très agréable et efficace !!,Date de l'expérience: 25 juillet 2022,25 juil. 2022,5,NaN,Réponse : Ohm Energie,https://fr.trustpilot.com/review/ohm-energie.com,trustpilot,positive
1474,1474,16,Excellent!!!,Excellent!!!! conseiller à l'écoute et prix at...,25 juil. 2022,5,"Bonjour,\nNous sommes ravis que nos services a...",Date de l'expérience: 25 juillet 2022,https://fr.trustpilot.com/review/ohm-energie.com,trustpilot,positive
1475,1475,16,Ohm énergie est très abordable.,Ohm énergie est très abordable.\nJe vous le re...,25 juil. 2022,5,"Bonjour,\nMerci pour votre retour d'expérience...",Date de l'expérience: 25 juillet 2022,https://fr.trustpilot.com/review/ohm-energie.com,trustpilot,positive


In [9]:
#Save in two variables the comments and the labels

sentences = reviews['verbatim']
labels = reviews['label']

## Loading the Pre-trained BERT model
Let's now load a pre-trained BERT model.

In [10]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Right now, the variable model holds a pretrained distilBERT model -- a version of BERT that is smaller, but much faster and requiring a lot less memory.

# Model 1: Preparing the Dataset
Before we can hand our sentences to BERT, we need to do some minimal processing to put them in the format it requires.

### Tokenization
Our first step is to tokenize the sentences -- break them up into word and subwords in the format BERT is comfortable with.

In [11]:
#Apply tokenization

# Import the necessary libraries
from transformers import AutoTokenizer

# Load the BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the sentences in the 'verbatim' column
tokenized = sentences.apply(lambda x: tokenizer.encode(x, add_special_tokens=True))


In [12]:
tokenized

0       [101, 15333, 6643, 2666, 5003, 2755, 5397, 200...
1       [101, 25540, 2666, 2755, 5397, 1037, 7367, 201...
2       [101, 6335, 2015, 2033, 2755, 5397, 3372, 7505...
3       [101, 4895, 2326, 7396, 15068, 6335, 9765, 424...
4       [101, 6581, 2063, 3325, 13642, 2278, 25540, 26...
                              ...                        
1472    [101, 5915, 4221, 1037, 16360, 15422, 2890, 10...
1473    [101, 3058, 2139, 1048, 1005, 3325, 1024, 2423...
1474    [101, 6581, 999, 999, 999, 999, 9530, 20240, 1...
1475    [101, 2821, 2213, 4372, 2121, 11239, 9765, 244...
1476    [101, 3058, 2139, 1048, 1005, 3325, 1024, 2423...
Name: verbatim, Length: 1000, dtype: object

### Padding
After tokenization, tokenized is a list of sentences -- each sentences is represented as a list of tokens. We want BERT to process our examples all at once (as one batch). It's just faster that way. For that reason, we need to pad all lists to the same size, so we can represent the input as one 2-d array, rather than a list of lists (of different lengths).


In [16]:
# Perform the padding

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

# Create a 2D array with the maximum length as the second dimension
padded = np.zeros((len(tokenized), max_len))

# Fill the array with the values from the original list of lists
for i, lst in enumerate(tokenized):
    padded[i, :len(lst)] = lst

print(padded)


[[  101. 15333.  6643. ...     0.     0.     0.]
 [  101. 25540.  2666. ...     0.     0.     0.]
 [  101.  6335.  2015. ...     0.     0.     0.]
 ...
 [  101.  6581.   999. ...     0.     0.     0.]
 [  101.  2821.  2213. ...     0.     0.     0.]
 [  101.  3058.  2139. ...     0.     0.     0.]]


Our dataset is now in the padded variable, we can view its dimensions below:

In [17]:
np.array(padded).shape

(1000, 226)

### Masking
If we directly send padded to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

In [18]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1000, 226)

## Model 1: And Now, Deep Learning!
Now that we have our model and inputs ready, let's run our model! 

The model() function runs our sentences through BERT. The results of the processing will be returned into last_hidden_states.

In [20]:
input_ids = torch.tensor(padded).long()  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

Let's slice only the part of the output that we need. That is the output corresponding the first token of each sentence. The way BERT does sentence classification, is that it adds a token called [CLS] (for classification) at the beginning of every sentence. The output corresponding to that token can be thought of as an embedding for the entire sentence.

We'll save those in the features variable, as they'll serve as the features to our logitics regression model.



In [21]:
features = last_hidden_states[0][:,0,:].numpy()

## Model 2: Train/Test Split
Let's now split our datset into a training set and testing set (even though we're using 1,000 sentences from the reviews training set).

In [22]:
# Split the data in train and test with the train_test_split function
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)


## [Bonus] Grid Search for Parameters
We can dive into Logistic regression directly with the Scikit Learn default parameters, but sometimes it's worth searching for the best value of the C parameter, which determines regularization strength.

In [26]:
# Instantiate a GridSearchCV object with the specified parameter grid and scoring metric
# Define the parameter grid to search over
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga']
}

grid_search = GridSearchCV(model, param_grid, scoring='accuracy')

# Fit the grid search object to the training data
grid_search.fit(X_train, y_train)

# Print the best combination of parameters and its corresponding score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Best parameters:  {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}
Best score:  0.905


We now train the LogisticRegression model. If you've chosen to do the gridsearch, you can plug the value of C into the model declaration (e.g. LogisticRegression(C=5.2)).

In [29]:
# Fit the logistic regression model
model = LogisticRegression()

# Fit the model to the training data
model.fit(X_train, y_train)

LogisticRegression()

## Evaluating Model 2
So how well does our model do in classifying sentences? One way is to check the accuracy against the testing dataset:

In [30]:
# Evaluate the score of the model
y_pred = model.predict(X_test)

# Evaluate the performance of the model on the test data
score = model.score(X_test, y_test)
print("R² score: ", score)

R² score:  0.94


How good is this score? What can we compare it against? Let's first look at a dummy classifier:

In [25]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

# COmpare your model with a dummy classifier

# Instantiate a DummyClassifier object with the "stratified" strategy and fit it to the training data
dummy = DummyClassifier(strategy="stratified")
dummy.fit(X_train, y_train)

# Use the dummy classifier to make predictions on the test data
dummy_pred = dummy.predict(X_test)

# Evaluate the performance of the dummy classifier on the test data
dummy_score = accuracy_score(y_test, dummy_pred)
print("Dummy score: ", dummy_score)

Dummy score:  0.44


So our model clearly does better than a dummy classifier